In [7]:
# use ggplot eventually
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import datetime as dt
from datetime import date

PATH_TO_DATA = '../data/'
PATH_TO_OUTPUT_DATA = '../output-data/'

In [2]:
MLB_GAME_DF_FINAL = pd.read_csv(PATH_TO_OUTPUT_DATA + 'GAME_LEVEL.csv')
MLB_GAME_DF_FINAL['dt'] = MLB_GAME_DF_FINAL['dt'].apply(lambda x:
                                                        dt.datetime.strptime(str(x), '%Y-%m-%d'))

Flow:

$$ELO_{Begin} = ELO_{Base} + Covariates$$ (538 model; Covariates do not have to have simple relationships
$$Compute_{ELO}$$ 
function updates ELOs

$$E_A=\frac{1}{1+10^{-(R_A - R_B)/200}}$$

$$R_A^{t+1}=R_A+16 \cdot(S_A-E_A)$$

Goal is to tune constants so that small differences in ELO don't impact too much

In [3]:
def compute_K(elo, i):
    return 40000/(2*elo + 3*i)

In [4]:
[compute_K(x, 160) for x in [1400, 1500, 1600]]

[12.195121951219512, 11.494252873563218, 10.869565217391305]

In [5]:
def initialize_elo_df(year, initial_values = []):
    ELO_STARTER_DF = MLB_GAME_DF_FINAL[['dt', 'Y']] # only use Y for filtering
    elo_df = ELO_STARTER_DF[ELO_STARTER_DF.Y == year].drop_duplicates().reset_index(drop = True)
    
    # baseline
    if (len(initial_values) == 0):
        for col in sorted(list(MLB_GAME_DF_FINAL.home_team.unique())):
            elo_df[col] = None
            elo_df.loc[0, col] = 1500 # just as a baseline
    else:
        for col in sorted(list(MLB_GAME_DF_FINAL.home_team.unique())):
            elo_df[col] = None
            elo_df.loc[0, col] = initial_values[col] # just as a baseline
    return elo_df.drop(columns = ['Y']).set_index('dt') # datetime index is easy!

In [1]:
# E_A, where A is the home team (so closer to 1 = chance home team wins)
def expected_elo_outcome(dt, home, away, elo_df, EXP_CONSTANT = 200):
    '''
        note that home and away are in AWAY @ HOME order in MLB_GAME_DF_FINAL
        dt = date of the game
        takes both teams current ELOs and computes
        check runtime!
    '''
    home_elo_prior = elo_df.loc[dt, home]
    away_elo_prior = elo_df.loc[dt, away]
    elo_expectation = (1 + 10**(-(home_elo_prior - away_elo_prior)/EXP_CONSTANT))**(-1) # equation works
    # function not used with compute_ELOs
    return elo_expectation

def compute_elo(dt, home, away, outcome, elo_df, i, K = 16, EXP_CONSTANT = 200):
    '''
        computes our new elos
        thoughts on K...scale based on time in season?
    '''
#     print()
    home_elo_prior = elo_df.loc[dt, home]
    away_elo_prior = elo_df.loc[dt, away]
    elo_expectation = (1 + 10**(-(home_elo_prior - away_elo_prior)/EXP_CONSTANT))**(-1) # equation works
#     print(str(home_elo_prior) + ' ' + str(away_elo_prior) + ' ' + str(elo_expectation))
#     expected_outcome = expected_elo_outcome(dt, away, home, elo_df)
#     print(outcome - elo_expectation)
    elo_delta = (outcome - elo_expectation)
    
    # if home team wins, Sa-Ea is positive; otherwise is negative and benefits away team
    home_elo_after = home_elo_prior + elo_delta * compute_K(home_elo_prior, i)
    away_elo_after = away_elo_prior - elo_delta * compute_K(home_elo_prior, i)
#     print(str(home_elo_after) + ' ' + str(away_elo_after) + ' ' + str(outcome))
    return [home_elo_after, away_elo_after, elo_expectation]

In [7]:
def compute_season_elos(year, initial_values = []):
    '''
        naive predictor based on the higher ELO
    '''
    # initialize data
    elo_YEAR = initialize_elo_df(year, initial_values)
    total_predictions = 0
    total_correct = 1
    
    for i in range(len(elo_YEAR)):    
        current_date = elo_YEAR.index[i]
        # uncaught try
        if (i + 1 == len(elo_YEAR)):
            break
        next_date = elo_YEAR.index[i + 1]
        # loop over all the matches that day
        RELEVANT_GAMES = MLB_GAME_DF_FINAL[MLB_GAME_DF_FINAL.dt == current_date].reset_index()
        for i in range(len(RELEVANT_GAMES)):
            row = RELEVANT_GAMES.iloc[i]
                
            new_elos = compute_elo(current_date, 
                                   row.home_team,
                                   row.away_team, 
                                   row.outcome, elo_YEAR, i)
            elo_YEAR.loc[next_date, row.home_team] = new_elos[0]
            elo_YEAR.loc[next_date, row.away_team] = new_elos[1]
            total_predictions = total_predictions + 1
            if (((new_elos[2] >= .5) & (row.outcome == 1)) | 
                ((new_elos[2] < .5) & (row.outcome == 0))):
                total_correct = total_correct + 1

            
#             if (row.home_team == 'SEA'):
#                 print(str(current_date) + ': ' + str(new_elos[0]) + ' ' + str(row.outcome))
#             if (row.away_team == 'SEA'):
#                 print(str(current_date) + ': '  + str(new_elos[1])  + ' ' + str(1 - row.outcome))
                
        # update remaining elos - ideally we have a better way to check this
        for col in elo_YEAR:
            if (elo_YEAR.loc[next_date, col] == None):
                elo_YEAR.loc[next_date, col] = elo_YEAR.loc[current_date, col]

    print(100 * total_correct/total_predictions) # accuracy
    return elo_YEAR

In [8]:
elo_2000_df = compute_season_elos(2000)

53.12888520513883


In [9]:
ELO_df = elo_2000_df.copy()
ELO_df.head()

,ANA,ARI,ATL,BAL,BOS,CHA,CHN,CIN,CLE,COL,...,PHI,PIT,SDN,SEA,SFN,SLN,TBA,TEX,TOR,WAS
dt,,,,,,,,,,,,,,,,,,,,,
2000-03-29,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,...,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500
2000-03-30,1500,1500,1500,1500,1500,1500,1506.67,1500,1500,1500,...,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500
2000-04-03,1500,1500,1500,1500,1500,1500,1499.52,1500,1500,1500,...,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500
2000-04-04,1493.41,1500,1506.66,1493.37,1500,1493.35,1492.92,1493.33,1506.63,1493.34,...,1500,1500,1493.41,1500,1493.4,1506.6,1506.64,1506.65,1506.65,1493.39
2000-04-05,1487.28,1506.6,1499.58,1493.37,1506.65,1487.24,1492.92,1487.2,1506.63,1500.42,...,1493.4,1493.37,1493.41,1493.35,1500.49,1506.6,1499.46,1512.76,1512.78,1487.24


In [10]:
# Append all ELOs
for year in range(2001, 2020):
    ELO_df = ELO_df.append(compute_season_elos(year, ELO_df.loc[max(ELO_df.index)]))

ELO_df.head()

55.40820555325321
55.7445043550394
56.52173913043478
56.44960597262547
54.34602649006622
53.23115161557581
51.668726823238565
53.03092783505155
54.324546952224054
53.46042271031911
54.74513054289267
55.36231884057971
53.5831960461285
51.74273858921162
53.5076795350768
53.524046434494196
52.91925465838509
56.9427276473012
57.510373443983404


,ANA,ARI,ATL,BAL,BOS,CHA,CHN,CIN,CLE,COL,...,PHI,PIT,SDN,SEA,SFN,SLN,TBA,TEX,TOR,WAS
dt,,,,,,,,,,,,,,,,,,,,,
2000-03-29,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,...,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500
2000-03-30,1500,1500,1500,1500,1500,1500,1506.67,1500,1500,1500,...,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500
2000-04-03,1500,1500,1500,1500,1500,1500,1499.52,1500,1500,1500,...,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500
2000-04-04,1493.41,1500,1506.66,1493.37,1500,1493.35,1492.92,1493.33,1506.63,1493.34,...,1500,1500,1493.41,1500,1493.4,1506.6,1506.64,1506.65,1506.65,1493.39
2000-04-05,1487.28,1506.6,1499.58,1493.37,1506.65,1487.24,1492.92,1487.2,1506.63,1500.42,...,1493.4,1493.37,1493.41,1493.35,1500.49,1506.6,1499.46,1512.76,1512.78,1487.24


In [25]:
ELOs = '''55.40820555325321
55.7445043550394
56.52173913043478
56.44960597262547
54.34602649006622
53.23115161557581
51.668726823238565
53.03092783505155
54.324546952224054
53.46042271031911
54.74513054289267
55.36231884057971
53.5831960461285
51.74273858921162
53.5076795350768
53.524046434494196
52.91925465838509
56.9427276473012
57.510373443983404'''

ELOs_list = [float(x)/100 for x in ELOs.split('\n')]

In [33]:
p = np.mean(ELOs_list)
print(p)
n = len(ELO_df)

0.5442228016715165


In [32]:
import math
SE = math.sqrt(p*(1-p)/n)
[p-1.96*SE, p+1.96*SE]

[0.5279917566990456, 0.5604538466439873]

In [30]:
len(ELO_df)

3617

In [222]:
ELO_df.to_csv(PATH_TO_OUTPUT_DATA + 'ELO_ratings.csv')

In [8]:
import pandas as pd
ELO_df = pd.read_csv(PATH_TO_OUTPUT_DATA + 'ELO_ratings.csv')

In [17]:
ELO_df.describe().T[['mean', 'min', 'max']].apply(lambda x: round(x*100)/100).T

,ANA,ARI,ATL,BAL,BOS,CHA,CHN,CIN,CLE,COL,...,PHI,PIT,SDN,SEA,SFN,SLN,TBA,TEX,TOR,WAS
mean,1516.86,1493.44,1507.95,1490.09,1525.03,1500.47,1496.99,1486.57,1505.65,1486.93,...,1498.28,1480.33,1486.05,1503.45,1501.97,1514.98,1498.65,1506.03,1504.31,1491.55
min,1436.30,1392.88,1386.65,1389.61,1434.07,1422.76,1407.66,1406.28,1418.50,1422.04,...,1404.77,1395.46,1398.87,1417.73,1417.64,1438.06,1401.77,1417.00,1434.12,1396.76
max,1596.05,1583.72,1591.91,1577.37,1600.92,1579.86,1594.62,1568.60,1614.38,1560.76,...,1581.33,1575.02,1574.69,1621.06,1597.85,1610.35,1581.84,1598.89,1578.42,1572.31
